## Практическое задание к уроку 9 по теме "Трансформер".

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

Загрузим библиотеки и датасеты:

In [1]:
import pandas as pd
import torch
from torch import nn
from torchinfo import summary
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

In [2]:
RANDOM_STATE = 29

In [3]:
df_train = pd.read_csv('../../Теория/Lesson_9/train.csv', index_col='id')
df_train.head()

,text,class
id,,
0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [4]:
df_val = pd.read_csv('../../Теория/Lesson_9/val.csv', index_col='id')

В качестве модели возьмём fine-tuned версию модели distilBERT:

In [5]:
model_name = 'philschmid/distilbert-base-multilingual-cased-sentiment-2'

model = pipeline('text-classification', model=model_name, device=0)

Сделаем предсказания на валидации. Модель возвращает метки трёх  
классов: позитивного, негативного и нейтрального. Нейтральный будем  
считать негативным, так как в результате теста этот способ показал  
себя лучше, чем если считать его позитивным.

In [6]:
def make_prediction(text):
    pred = model(text)[0]['label']
    match pred:
        case 'positive':
            pred = 1
        case 'negative':
            pred = 0
        case 'neutral':
            pred = 0
    return pred

In [7]:
tqdm.pandas()

predictions = df_val['text'].progress_apply(lambda x: make_prediction(x))

  0%|                                       | 2/22683 [00:00<1:22:03,  4.61it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|████████████████████████████████████| 22683/22683 [01:13<00:00, 308.05it/s]


Оценим точность предсказаний:

In [8]:
(predictions == df_val['class']).sum() / len(df_val)

0.8086231979896839

Результат составил почти 81%, что заметно превышает результаты  
моделей CNN и RNN, полученные на предыдущих уроках.  
Теперь дообучим эту модель под наш датасет на два класса.  
Сначала обернём датасет в даталоадер. В процессе будем делать  
токенизацию, как делали на уроке.

In [9]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self._txts = [self.tokenizer(text, padding='max_length', max_length=70,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [10]:
torch.random.manual_seed(RANDOM_STATE)

y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=32,
                          shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=32,
                          shuffle=False)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Посмотрим на модель. При загрузке укажем, что у нас будет два класса, а не три:

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at philschmid/distilbert-base-multilingual-cased-sentiment-2 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

Будем обучать два последних линейных слоя. Посмотрим на количество параметров:

In [13]:
summary(model)

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              91,812,096
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 135,326,210
Trainable params: 135,326,210
Non-trainable params: 0

Обучить нужно примерно 592 тыс. параметров из 135 млн.  
Проведём обучение:

In [14]:
torch.random.manual_seed(RANDOM_STATE)
torch.backends.cudnn.deterministic = True

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
model = model.to(device)

param_groups = [
    {'params': model.pre_classifier.parameters(), 'lr': 0.001},
    {'params': model.classifier.parameters(), 'lr': 0.001}
]
optimizer = torch.optim.Adam(param_groups)

for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    for train_input, train_label in tqdm(train_loader):
        model.train()
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_ids=input_id, attention_mask=mask, labels=train_label)
                
        batch_loss = output.loss
        total_loss_train += batch_loss
        
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
        acc = (output.logits.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    with torch.no_grad():
        for val_input, val_label in valid_loader:
            val_label = val_label.to(device)
            mask = val_input['attention_mask'].to(device)
            input_id = val_input['input_ids'].squeeze(1).to(device)

            output = model(input_ids=input_id, attention_mask=mask, labels=val_label)

            batch_loss = output.loss
            total_loss_val += batch_loss

            acc = (output.logits.argmax(dim=1) == val_label).sum().item()
            total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_loader): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_loader): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at philschmid/distilbert-base-multilingual-cased-sentiment-2 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|███████████████████████████████████████| 5671/5671 [08:39<00:00, 10.92it/s]


Epochs: 1 | Train Loss:  0.423         | Train Accuracy:  0.804         | Val Loss:  0.384         | Val Accuracy:  0.826


100%|███████████████████████████████████████| 5671/5671 [08:03<00:00, 11.74it/s]


Epochs: 2 | Train Loss:  0.412         | Train Accuracy:  0.811         | Val Loss:  0.368         | Val Accuracy:  0.833


Дообучение позволило увеличить точность модели на 2.5%.  
Можно ещё добавить эпох и поподбирать гиперпараметры  
оптимизатора, но одна эпоха обучается 8 минут, и подбор  
займёт прилично времени. Считаю задание выполненным.